In [136]:
import pandas as pd

In [137]:
from gensim.models import word2vec
from sklearn.manifold import TSNE
import plotly_express as px

In [146]:
LIB = pd.read_csv('/home/yk4we/ETA_final.LIB.csv')
DOC = pd.read_csv('/home/yk4we/ETA_final.DOC.csv')
TOKEN = pd.read_csv('/home/yk4we/ETA_final.TOKEN.csv')
VOCAB = pd.read_csv('/home/yk4we/ETA_final.VOCAB.csv')

In [151]:
#remove stop words
VOCAB = VOCAB.loc[VOCAB.stop == 0]

In [155]:
#remove stop words from TOKEN table
TOKEN = TOKEN.loc[TOKEN.term_id.isin(VOCAB.term_id)]

In [156]:
powerline = LIB.loc[LIB.doc_source.str.contains('PowerLine')].doc_id.to_list()
daily = LIB.loc[LIB.doc_source.str.contains('Daily')].doc_id.to_list()

In [157]:
TOKEN_powerline = TOKEN.loc[TOKEN.doc_id.isin(powerline)]
TOKEN_daily= TOKEN.loc[TOKEN.doc_id.isin(daily)]

In [158]:
TOKEN_powerline['source'] = 'powerline'
TOKEN_daily['source'] = 'daily'

/apps/software/standard/compiler/gcc/7.1.0/jupyter_conda/2019.10-py3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/apps/software/standard/compiler/gcc/7.1.0/jupyter_conda/2019.10-py3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [159]:
TOKEN_powerline['doc'] = TOKEN_powerline.doc_id.map(LIB.reset_index().set_index('doc_id').doc_title)
TOKEN_daily['doc'] = TOKEN_daily.doc_id.map(LIB.reset_index().set_index('doc_id').doc_title)

/apps/software/standard/compiler/gcc/7.1.0/jupyter_conda/2019.10-py3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/apps/software/standard/compiler/gcc/7.1.0/jupyter_conda/2019.10-py3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [160]:
TOKEN_powerline.tail()

,doc_id,sent_num,token_num,pos_tuple,pos,token_str,term_str,term_id,source,doc
8586601,1026333,17,17,"('billions', 'NNS')",NNS,billions,billions,17228,powerline,Omar Slanders a Much Better Person
8586603,1026333,17,19,"('people', 'NNS')",NNS,people,people,78439,powerline,Omar Slanders a Much Better Person
8586604,1026333,17,20,"('represents', 'VBZ')",VBZ,represents,represents,89116,powerline,Omar Slanders a Much Better Person
8586606,1026333,17,22,"('new', 'JJ')",JJ,new,new,71385,powerline,Omar Slanders a Much Better Person
8586607,1026333,17,23,"('low', 'JJ')",JJ,low,low,63296,powerline,Omar Slanders a Much Better Person


In [161]:
TOKEN_daily.head()

,doc_id,sent_num,token_num,pos_tuple,pos,token_str,term_str,term_id,source,doc
1588,96,0,0,"('Income', 'NNP')",NNP,Income,income,53898,daily,Obama has done nothing to address income inequ...
1590,96,0,2,"('wealth', 'NN')",NN,wealth,wealth,114955,daily,Obama has done nothing to address income inequ...
1591,96,0,3,"('inequality', 'NN')",NN,inequality,inequality,54305,daily,Obama has done nothing to address income inequ...
1594,96,0,6,"('festering', 'VBG')",VBG,festering,festering,41492,daily,Obama has done nothing to address income inequ...
1595,96,0,7,"('wound', 'NN')",NN,wound,wound,117290,daily,Obama has done nothing to address income inequ...


In [162]:
OHCO = ['source','doc', 'sent_num', 'token_num']
BAG = OHCO[:1] # Sentence

In [163]:
TOKEN_powerline = TOKEN_powerline.set_index(OHCO)
TOKEN_daily = TOKEN_daily.set_index(OHCO)

In [164]:
 #remove proper noun, plural
corpus_powerline = TOKEN_powerline [~TOKEN_powerline .pos.str.match('NNPS?')]\
    .groupby(BAG)\
    .term_str.apply(lambda  x:  x.astype(str).tolist())\
    .reset_index()['term_str'].tolist()

In [165]:
corpus_daily = TOKEN_daily[~TOKEN_daily.pos.str.match('NNPS?')]\
    .groupby(BAG)\
    .term_str.apply(lambda  x:  x.tolist())\
    .reset_index()['term_str'].tolist()

## Generate Word Embeddings wigh Gensim's Library

In [166]:
window = 5

In [167]:
model_powerline = word2vec.Word2Vec(corpus_powerline, size=246, window=window, min_count=200, workers=4)

In [168]:
model_daily = word2vec.Word2Vec(corpus_daily, size=246, window=window, min_count=200, workers=4)

### Visualize with tSNE

#### Generate coordinate to Plot

In [169]:
coords_powerline = pd.DataFrame(index=range(len(model_powerline.wv.vocab)))
coords_powerline['label'] = [w for w in model_powerline.wv.vocab]
coords_powerline['vector'] = coords_powerline['label'].apply(lambda x: model_powerline.wv.get_vector(x))

In [170]:
coords_powerline.head()

,label,vector
0,behind,"[-0.001130699, 0.0021639834, 0.0018210007, 0.0..."
1,many,"[-0.0031521257, 0.0034727799, -0.00054585165, ..."
2,regular,"[0.0010632193, 0.0013396019, 6.475637e-05, 0.0..."
3,days,"[-2.2584078e-05, 0.00045350654, -0.0010550807,..."
4,among,"[-0.0009490203, 0.0010769675, 0.0019000672, 0...."


In [171]:
coords_daily = pd.DataFrame(index=range(len(model_daily.wv.vocab)))
coords_daily['label'] = [w for w in model_daily.wv.vocab]
coords_daily['vector'] = coords_daily['label'].apply(lambda x: model_daily.wv.get_vector(x))

In [172]:
coords_daily.head()

,label,vector
0,wealth,"[-0.00051380374, 0.0008119143, -0.00050975726,..."
1,society,"[0.00074736314, 0.0013763912, 0.0014489185, 0...."
2,worse,"[-0.00017124719, 0.00039373385, 0.0002663248, ..."
3,past,"[-0.0042202845, -0.00059243705, 0.0011397752, ..."
4,five,"[-0.00034998584, 0.00033491416, 0.0013233055, ..."


#### Use Scikit-learn's TSNE library

In [173]:
tsne_model_powerline = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
tsne_values_powerline = tsne_model_powerline.fit_transform(coords_powerline['vector'].tolist())

In [174]:
coords_powerline['x'] = tsne_values_powerline[:,0]
coords_powerline['y'] = tsne_values_powerline[:,1]

In [175]:
coords_powerline.head()

,label,vector,x,y
0,behind,"[-0.001130699, 0.0021639834, 0.0018210007, 0.0...",40.366013,0.079792
1,many,"[-0.0031521257, 0.0034727799, -0.00054585165, ...",53.649952,-3.910273
2,regular,"[0.0010632193, 0.0013396019, 6.475637e-05, 0.0...",-9.228317,8.140360
3,days,"[-2.2584078e-05, 0.00045350654, -0.0010550807,...",39.829773,-3.928443
4,among,"[-0.0009490203, 0.0010769675, 0.0019000672, 0....",34.011681,-10.588156


In [176]:
tsne_model_daily = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
tsne_values_daily = tsne_model_daily.fit_transform(coords_daily['vector'].tolist())

In [177]:
coords_daily['x'] = tsne_values_daily[:,0]
coords_daily['y'] = tsne_values_daily[:,1]

In [178]:
coords_daily.head()

,label,vector,x,y
0,wealth,"[-0.00051380374, 0.0008119143, -0.00050975726,...",15.579317,-4.409695
1,society,"[0.00074736314, 0.0013763912, 0.0014489185, 0....",-16.863220,-4.485227
2,worse,"[-0.00017124719, 0.00039373385, 0.0002663248, ...",-9.262919,3.558114
3,past,"[-0.0042202845, -0.00059243705, 0.0011397752, ...",51.892834,0.604571
4,five,"[-0.00034998584, 0.00033491416, 0.0013233055, ...",4.927863,17.844521


#### Plot Coordinates

In [179]:
fig = px.scatter(coords_powerline, 'x', 'y', text='label', height=1000).update_traces(mode='text')

In [180]:
fig.write_html("/home/yk4we/ETA_final/fig_powerline.html")

## Semantic Algebra

### Analogy

$A : B :: C : D? \rightarrow B - A + C = D$

In [181]:
def complete_analogy_powerline(A, B, C, n=2):
    try:
        return model_powerline.wv.most_similar(positive=[B, C], negative=[A])[0:n]
    except KeyError as e:
        print('Error:', e)
        return None

In [210]:
complete_analogy_powerline('trump', 'republican', 'candidate')

[('width', 0.35304051637649536), ('make', 0.3314433991909027)]

In [184]:
def complete_analogy_daily(A, B, C, n=2):
    try:
        return model_daily.wv.most_similar(positive=[B, C], negative=[A])[0:n]
    except KeyError as e:
        print('Error:', e)
        return None

In [204]:
complete_analogy_daily('trump', 'trump', 'trump')

[('sending', 0.2238604575395584), ('conservative', 0.20739640295505524)]

In [191]:
model_powerline.wv.most_similar('republican')

[('report', 0.5620759725570679),
 ('debate', 0.5468108654022217),
 ('one', 0.5462952852249146),
 ('administration', 0.5359621047973633),
 ('many', 0.5309634208679199),
 ('good', 0.5268799066543579),
 ('us', 0.5261294841766357),
 ('say', 0.5223155617713928),
 ('last', 0.5183826088905334),
 ('two', 0.5144458413124084)]

In [201]:
model_daily.wv.most_similar('republican')

[('nt', 0.8552619814872742),
 ('percent', 0.8515831828117371),
 ('one', 0.8448664546012878),
 ('insurance', 0.8440598249435425),
 ('time', 0.8435603380203247),
 ('food', 0.843031644821167),
 ('support', 0.8411489725112915),
 ('even', 0.8399174213409424),
 ('coverage', 0.8389263153076172),
 ('health', 0.8385680317878723)]